<a href="https://colab.research.google.com/github/restful3/ds4th_study/blob/main/source/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9D%84%20%ED%99%9C%EC%9A%A9%ED%95%9C%20%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%20%EC%BF%A1%EB%B6%81/5_%EB%B2%94%EC%A3%BC%ED%98%95%EB%8D%B0%EC%9D%B4%ED%84%B0_ws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5 범주형 데이터 다루기
- 관측대상을 양이 아닌 질로 측정할 때가 유용할 때가 있음
- 일부범주는 자연적 순서를 지님 (순서형 범주)
    - 낮음/중간/높음, 청년/노인, 동의/중립/반대 등
- 종종 벡터나 문자열로 표현

## 5.1 순서가 없는 범주형 데이터 인코딩하기
- 순서를 가지지 않는 클래스는 레이블인코딩이나 원핫인코딩 처리
- LabelBinarizer는 문자열 타깃데이터를 원핫인코딩으로 변환시 사용
    - 정수 레이블로 변환할 때는 LableEncoder

[참고: 더미변수의 함정](http://bit.ly/2FvVJkC)<br>
[참고: 원핫인코딩사용할때의 열삭제](https://stats.stackexchange.com/questions/231285/dropping-one-of-the-columns-when-using-one-hot-encoding)

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import warnings
warnings.filterwarnings('ignore')

feature = np.array([['Texas'],
                    ['California'],
                    ['Texas'],
                    ['Delaware'],
                    ['Texas']])

onehot = LabelBinarizer()
onehot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [ ]:
onehot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

In [ ]:
onehot.inverse_transform(onehot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

- pandas 활용한 원핫인코딩

In [ ]:
import pandas as pd
pd.get_dummies(feature[:,0], dtype=int, drop_first=True)

,Delaware,Texas
0,0,1
1,0,0
2,0,1
3,1,0
4,0,1


- 사이킷런의 유용한 기능은 샘플이 여러개의 클래스 가지고 있는 경우를 다룰수 있다는 것

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
multi_feature= [('Texas','Florida'),
               ('California','Alabama'),
               ('Texas','Florida'),
               ('Delaware','Florida'),
               ('Texas','Alabama')]
onehot_multiclass = MultiLabelBinarizer()
onehot_multiclass.fit_transform(multi_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [ ]:
onehot_multiclass.classes_

array(['Alabama', 'California', 'Delaware', 'Florida', 'Texas'],
      dtype=object)

- OneHotEncoder, 여러개의 열이 있는 특성배열 생성을 인코딩할 때 사용
    - sparse = False로 지정하면 밀집 배열을 얻을 수 있음

In [ ]:
from sklearn.preprocessing import OneHotEncoder
warnings.filterwarnings('ignore')

feature = np.array([['Texas',1],
                    ['California',1],
                    ['Texas',3],
                    ['Delaware',1],
                    ['Texas',1]])
onehot_encoder = OneHotEncoder(sparse=False) # 밀집베열
onehot_encoder.fit_transform(feature)

array([[0., 0., 1., 1., 0.],
       [1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1.],
       [0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 0.]])

In [ ]:
onehot_encoder.categories_

[array(['California', 'Delaware', 'Texas'], dtype='<U11'),
 array(['1', '3'], dtype='<U11')]

- 판다스로 데이터프레임도 원핫인코딩 가능

In [ ]:
pd.get_dummies(pd.DataFrame(feature), dtype=int, drop_first=True)

,0_Delaware,0_Texas,1_3
0,0,1,0
1,0,0,0
2,0,1,1
3,1,0,0
4,0,1,0


## 5.2 순서가 있는 범주형 데이터 인코딩하기

In [ ]:
import pandas as pd
df = pd.DataFrame({'Score':['Low','Low','Medium','Medium','High']})

scale_mapper = {'Low':1,
                'Medium':2,
                'High':3}

# 딕셔너리로 맵핑하기
df['Score'].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

In [ ]:
# 스케일 크기 지정에 주의 필요
df = pd.DataFrame({'Score':['Low','Low','Medium','Medium','High',
                            'Barely More Than Medium']})

scale_mapper = {'Low':1,
                'Medium':2,
                'Barely More Than Medium':2.1,
                'High':3}

# 딕셔너리로 맵핑하기
df['Score'].replace(scale_mapper)

0    1.0
1    1.0
2    2.0
3    2.0
4    3.0
5    2.1
Name: Score, dtype: float64

In [ ]:
df['Score'].map(scale_mapper)

0    1.0
1    1.0
2    2.0
3    2.0
4    3.0
5    2.1
Name: Score, dtype: float64

- 클래스 범주를 순서대로 변환 : OrdinalEncoder
    - 책에는 나오지만 쓰지않는 것이 나아보임

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
feature=np.array([['Low',3],
                  ['High',2],
                  ['Medium',5]])
ordinar_encoder = OrdinalEncoder(dtype=int)
ordinar_encoder.fit_transform(feature)


array([[1, 1],
       [0, 0],
       [2, 2]])

In [ ]:
ordinar_encoder.categories_

[array(['High', 'Low', 'Medium'], dtype='<U11'),
 array(['2', '3', '5'], dtype='<U11')]

## 5.3 특성 딕셔너리를 인코딩하기
- 딕셔너리를 특성행렬로 변환하기
- 특성딕셔너리를 인코딩하는 것은 자연어처리 분야에서 주로 쓰임.
    - 예> 문서데이터 가지고 있을 때 각 문서에 등장한 모든 단어의 횟수 담은 딕셔너리 데이터 생성

In [ ]:
from sklearn.feature_extraction import DictVectorizer
data_dict = [{'Red':2,'Blue':4},
             {'Red':4,'Blue':3},
             {'Red':1,'Yellow':2},
             {'Red':2,'Yellow':2}]
dictvectorizer = DictVectorizer(sparse=False)
feature = dictvectorizer.fit_transform(data_dict)
feature


array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

In [ ]:
pd.DataFrame(feature,columns=dictvectorizer.feature_names_)

,Blue,Red,Yellow
0,4.0,2.0,0.0
1,3.0,4.0,0.0
2,0.0,1.0,2.0
3,0.0,2.0,2.0


## 5.4 결측치 대체하기
- 범주형 데이터의 결측치도 KNN으로 대체 가능

X로 훈련시킨 k이웃모델로 결측치를 예측해본다

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# 훈련용 데이터
X= np.array([[0,2.1,1.45],
             [1,1.18,1.33],
             [0,1.22,1.27],
             [1,-0.21,-1.19]])

# 예측용 데이터
X_with_nan = np.array([[np.nan,0.87,1.31],
                       [np.nan,-0.67,0.22]])

# 모델 생성
clf=KNeighborsClassifier(3, weights='distance')

# 훈련 : 1~2열의 데이터로 0열의 특성을 예측하는 훈련
trained_model = clf.fit(X[:,1:], X[:,0])

# 예측 : 0열의 예측값 담아줌
imputed_values = trained_model.predict(X_with_nan[:,1:])

# 예측, 원본특성 결합
X_with_imputed = np.hstack((imputed_values.reshape(-1,1), X_with_nan[:,1:]))

# 예측용데이터 + 훈련용데이터 연결
np.vstack((X_with_imputed,X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67,  0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

- 최빈값으로 채우기

In [ ]:
from sklearn.impute import SimpleImputer

X_complete = np.vstack((X_with_nan,X))

imputer = SimpleImputer(strategy='most_frequent')

imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67,  0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

- !! 중요
    - 책에는 설명이 생략되었으나, 데이터의 성격에 따라 결측치 자체가 하나의 범주로 바꿔서 처리되기도 함 (결측치가 발생한 원인이 있을 수도 있으니...)
    - 결측치가 많지 않을 경우, 생략되기도 함

## 5.5 불균형한 클래스 다루기
- 실전에는 불균형한 데이터가 매우 많기에 불균형한 클래스를 다루는 일은 자주 발생함
    - 소수클래스 샘플을 아주 많이 모으거나
    - 불균형한 클래스에 잘맞는 모델평가지표를 사용 (재현율고려, f1스코어, roc곡선)
    - 클래스가중치매개변수 사용(class_weight 등)
    - 다운샘플링/업샘플링 기법 사용

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

iris = load_iris()
feature = iris.data
target = iris.target

# 처음 40개 샘플 삭제조치
feature = feature[40:,:]
target = target[40:]

# 클래스0을 음성클래스로 하는 불균형한 이진타겟벡터생성
target = np.where((target==0),0,1)
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

- 사이킷런의 많은 알고리즘은 훈련시 불균형 영향을 줄일 수 있도록 클래스에 가중치를 부여할 수 있는 매개변수를 제공
    - randomforest는 class_weight로 조정

In [ ]:
#가중치 만들고
weight ={0:.9,1:.1}
# 가중치 부여한 분류기 생성
rfc = RandomForestClassifier(class_weight=weight)
# RandomForestClassifier(class_weight='balanced') # 균형잡힌 가중치로 모델 훈련
rfc.fit(feature,target)

RandomForestClassifier(class_weight={0: 0.9, 1: 0.1})

- 다수클래스의 샘플수 줄이거나(다운샘플링)


In [ ]:
# 각 클래스의 샘플인덱스 추출
i_class0 = np.where(target==0)[0]
i_class1 = np.where(target==1)[0]

# 각 클래스의 샘플수
n_class0 = len(i_class0)
n_class1 = len(i_class1)

# 클래스 0의 샘플만큼 클래스 1에서 중복을 허용하지 않고 랜덤하게 샘플추출
i_class1_downsampled = np.random.choice(i_class1, size=n_class0,
                                        replace= False)

# 클래스 0의 타깃벡터와 다운샘플링된 클래스1의 타깃백터를 합침
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# 클래스 0의 특성행렬와 다운샘플링된 클래스1의 특성행렬 합침
np.vstack((feature[i_class0,:], feature[i_class1_downsampled,:]))

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4],
       [4.8, 3. , 1.4, 0.3],
       [5.1, 3.8, 1.6, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [5. , 3.3, 1.4, 0.2],
       [6.9, 3.1, 5.4, 2.1],
       [6.8, 3. , 5.5, 2.1],
       [4.9, 2.5, 4.5, 1.7],
       [6.8, 2.8, 4.8, 1.4],
       [6.5, 2.8, 4.6, 1.5],
       [5.8, 2.7, 5.1, 1.9],
       [7.7, 2.8, 6.7, 2. ],
       [5.5, 2.6, 4.4, 1.2],
       [6.7, 3.1, 4.7, 1.5],
       [5.5, 2.3, 4. , 1.3]])

- 소수클래스의 샘플수 늘릴수도 (업샘플링)

In [ ]:
i_class0_upsampled = np.random.choice(i_class0, size=n_class1,replace= True)
np.hstack((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [ ]:
np.vstack((feature[i_class0_upsampled,:], feature[i_class1,:]))[:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.3, 0.3],
       [5. , 3.5, 1.6, 0.6],
       [4.8, 3. , 1.4, 0.3]])